In [1]:
from pathlib import Path

import numba as nb
import numpy as np
import scipy.stats as stats

from natsort import natsorted

import matplotlib.pyplot as plt


from threeML import *

#plt.style.use("thomas")
import warnings
warnings.simplefilter("ignore")


%matplotlib widget



Welcome to JupyROOT 6.22/02
[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available
[INFO    ] Starting 3ML!
[WARNING ] Multinest minimizer not available
[WARNING ] PyGMO is not available
[WARNING ] The cthreeML package is not installed. You will not be able to use plugins which require the C/C++ interface (currently HAWC)
[WARNING ] Could not import plugin HAWCLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Could not import plugin FermiLATLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal performances in 3ML
[WARNING ] Env. variable MKL_NUM_THREADS is not set. Please set it to 1 for opti

In [2]:
spi = OGIPLike("spi",
               observation="fit_GRB120711A_BB_SwiftPosition_spec/spectra_GRB_OGIP.fits",
               response="fit_GRB120711A_BB_SwiftPosition_spec/spectral_response.rmf_OGIP.fits")



[WARNING ] Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.
[WARNING ] FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.
[WARNING ] The response is in an extension called MATRIX, which usually means you also need an ancillary file (ARF) which you didn't provide. You should refer to the documentation  of the instrument and make sure you don't need an ARF.
[INFO    ] Auto-probed noise models:
[INFO    ] - observation: gaussian
[INFO    ] - background: None


In [3]:
spi.view_count_spectrum();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
"""band = Band()
band.alpha.prior = Truncated_gaussian(lower_bound=-1.5, upper_bound=1, mu=-1, sigma=0.5)
band.beta.prior = Truncated_gaussian(lower_bound=-5, upper_bound=-1.6, mu=-2, sigma=0.5)
band.xp.prior = Log_normal(mu=2, sigma=1)
band.xp.bounds = (0, None)
band.K.prior = Log_uniform_prior(lower_bound=1e-10, upper_bound=1e3)
ps = PointSource("grb", 0, 0, spectral_shape=band)
band_model = Model(ps)"""


band = Band()
band.alpha.min_value = None
band.alpha.max_value = None
band.alpha.prior = Truncated_gaussian(lower_bound=-1.5, upper_bound=10, mu=-1, sigma=1)

band.beta.min_value = None
band.beta.max_value = None
band.beta.prior = Truncated_gaussian(lower_bound=-10, upper_bound=-1.6, mu=-4, sigma=2)
#band.xp.prior = Log_normal(mu=2, sigma=1)
band.xp.prior = Truncated_gaussian(lower_bound=25, upper_bound=8000, mu=1000, sigma=1000)
band.xp.bounds = (None, None)
#band.K.prior = Log_uniform_prior(lower_bound=1e-10, upper_bound=1e3)
band.K.prior = Truncated_gaussian(lower_bound=1e-6, upper_bound=1e1, mu=5e-2, sigma=1e-1)
ps = PointSource("grb", 0, 0, spectral_shape=band)
band_model = Model(ps)

In [5]:
bayes = BayesianAnalysis(band_model, DataList(spi))

In [6]:
#bayes.set_sampler("multinest")
#bayes.sampler.setup()#n_live_points=400)
#_ = bayes.sample()

bayes.set_sampler()
bayes.sampler.setup(n_iterations=2500,n_burn_in=2500,n_walkers=20)
_ = bayes.sample()

[INFO    ] using default sampler emcee
[INFO    ] sampler is setup with default parameters


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

[INFO    ] Mean acceptance fraction: 0.57008
Maximum a posteriori probability (MAP) point:



,result,unit
parameter,,
grb.spectrum.main.Band.K,(2.95 +/- 0.10) x 10^-2,1 / (cm2 keV s)
grb.spectrum.main.Band.alpha,(-9.4 +/- 0.5) x 10^-1,
grb.spectrum.main.Band.xp,(1.51 -0.24 +0.25) x 10^3,keV
grb.spectrum.main.Band.beta,-5.2 -2.2 +2.1,



Values of -log(posterior) at the minimum:



,-log(posterior)
spi,-16.074467
total,-16.074467



Values of statistical measures:



,statistical measures
AIC,41.630416
BIC,46.011878
DIC,39.187907
PDIC,3.422041


In [7]:
bayes.restore_median_fit()

display_spectrum_model_counts(bayes);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plot_spectra(bayes.results, flux_unit="keV2/(cm2 s keV)", num_ene=100);

processing MLE analyses: 0it [00:00, ?it/s]

processing Bayesian analyses:   0%|          | 0/1 [00:00<?, ?it/s]

Propagating errors:   0%|          | 0/100 [00:00<?, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
sigma = 68.3
bayes.results.corner_plot(
                    #truths=np.array([1,2,3,4]),
                    quantiles=(50+sigma/np.array([-2,np.inf,+2]))/100.,
                    #quantiles=(0.16, 0.50, 0.84),
                    show_titles=True,
                    bins=25,
                    fill_contours=True,
                    contourf_kwargs={"cmap": plt.cm.viridis, "colors":None},
                    levels=[1-np.exp(-4.5),1-np.exp(-2.0),1-np.exp(-0.5)],
                    truth_color='orange');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
bayes.results.get_data_frame("equal tail")

,value,negative_error,positive_error,error,unit
grb.spectrum.main.Band.K,0.029511,-0.000961,0.000981,0.000971,1 / (cm2 keV s)
grb.spectrum.main.Band.alpha,-0.939658,-0.046424,0.046631,0.046527,
grb.spectrum.main.Band.xp,1510.474295,-244.749812,245.803922,245.276867,keV
grb.spectrum.main.Band.beta,-5.172231,-2.240492,2.145184,2.192838,


In [11]:
bayes.results.get_data_frame("hpd")

,value,negative_error,positive_error,error,unit
grb.spectrum.main.Band.K,0.029511,-0.001018,0.000919,0.000968,1 / (cm2 keV s)
grb.spectrum.main.Band.alpha,-0.939658,-0.049488,0.042878,0.046183,
grb.spectrum.main.Band.xp,1510.474295,-260.159041,228.573207,244.366124,keV
grb.spectrum.main.Band.beta,-5.172231,-1.321268,2.789043,2.055156,


In [12]:
ar2 = bayes.results

In [13]:
np.sqrt(np.diag(ar2.estimate_covariance_matrix()))

array([1.00500938e-03, 4.70418612e-02, 2.53051335e+02, 1.99388557e+00])

In [14]:
np.mean(ar2.samples[0,:]),np.std(ar2.samples[0,:])

(0.029511279744822833, 0.0010049993321096468)